In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import SimpleImputer
# Load organizations dataset
df = pd.read_csv("organizations-100.csv")

# Initial sanity checks
print("Dataset Shape:", df.shape)
print("\nFirst 5 Rows:")
print(df.head())

print("\nDataset Info:")
df.info()

print("\nStatistical Summary:")
print(df.describe(include="all"))
# Check duplicate rows
print("\nDuplicate rows:", df.duplicated().sum())

# Remove duplicate rows
df = df.drop_duplicates()

# Check unique organization IDs
if "organization_id" in df.columns:
    print("Unique Organization IDs:", df["organization_id"].nunique())
# Drop irrelevant or temporary columns
drop_cols = ["temp_id", "notes", "comments"]
df.drop(columns=drop_cols, inplace=True, errors="ignore")

# Rename columns for clarity
df.rename(columns={
    "org_id": "organization_id",
    "emp_cnt": "employee_count",
    "web": "website"
}, inplace=True)

# Reorder columns alphabetically
df = df.reindex(sorted(df.columns), axis=1)

print("\nColumns after management:")
print(df.columns)
# Check missing values
print("\nMissing values per column:")
print(df.isna().sum())

# Drop columns with >70% missing values
threshold = 0.7 * len(df)
df = df.dropna(axis=1, thresh=threshold)
# Identify numeric & categorical columns
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
cat_cols = df.select_dtypes(include=["object"]).columns

# Impute employee_count using median (robust to outliers)
if "employee_count" in df.columns:
    emp_imputer = SimpleImputer(strategy="median")
    df[["employee_count"]] = emp_imputer.fit_transform(df[["employee_count"]])

# Impute remaining numeric columns
num_imputer = SimpleImputer(strategy="median")
df[num_cols] = num_imputer.fit_transform(df[num_cols])

# Impute categorical columns using mode
cat_imputer = SimpleImputer(strategy="most_frequent")
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])
# Convert founded_year to numeric
if "founded_year" in df.columns:
    df["founded_year"] = pd.to_numeric(df["founded_year"], errors="coerce")

# Convert employee_count to integer
if "employee_count" in df.columns:
    df["employee_count"] = df["employee_count"].astype(int)

# Convert categorical columns to category dtype
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype("category")
# Normalize industry text
if "industry" in df.columns:
    df["industry"] = df["industry"].str.lower().str.strip()

    industry_map = {
        "it services": "information technology",
        "it": "information technology",
        "software": "information technology",
        "fintech": "financial services",
        "finance": "financial services",
        "health care": "healthcare",
        "health-care": "healthcare"
    }

    df["industry"] = df["industry"].replace(industry_map)
def clean_website(url):
    if pd.isna(url):
        return np.nan

    url = url.strip().lower()

    # Add scheme if missing
    if not url.startswith(("http://", "https://")):
        url = "https://" + url

    # Basic URL validation
    pattern = r"https?://[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    if re.match(pattern, url):
        return url
    else:
        return np.nan

if "website" in df.columns:
    df["website"] = df["website"].apply(clean_website)
# Normalize all categorical text columns
for col in df.select_dtypes(include="category").columns:
    df[col] = df[col].astype(str).str.lower().str.strip()
# Ensure no duplicates remain
assert df.duplicated().sum() == 0, "Duplicate rows still exist!"

print("\nFinal Missing Values:")
print(df.isna().sum())

print("\nFinal Dataset Info:")
df.info()
# Ensure no duplicates remain
assert df.duplicated().sum() == 0, "Duplicate rows still exist!"

print("\nFinal Missing Values:")
print(df.isna().sum())

print("\nFinal Dataset Info:")
df.info()
# Save cleaned organizations dataset
df.to_csv("cleaned_organizations_data.csv", index=False)

print("\n✅ Organizations dataset cleaned successfully!")
print("📁 Saved as: cleaned_organizations_data.csv")


Dataset Shape: (100, 9)

First 5 Rows:
   Index  Organization Id                     Name  \
0      1  FAB0d41d5b5d22c              Ferrell LLC   
1      2  6A7EdDEA9FaDC52  Mckinney, Riley and Day   
2      3  0bFED1ADAE4bcC1               Hester Ltd   
3      4  2bFC1Be8a4ce42f           Holder-Sellers   
4      5  9eE8A6a4Eb96C24              Mayer Group   

                          Website           Country  \
0              https://price.net/  Papua New Guinea   
1  http://www.hall-buchanan.info/           Finland   
2       http://sullivan-reed.com/             China   
3             https://becker.com/      Turkmenistan   
4          http://www.brewer.com/         Mauritius   

                                      Description  Founded  \
0             Horizontal empowering knowledgebase     1990   
1             User-centric system-worthy leverage     2015   
2                  Switchable scalable moratorium     1971   
3  De-engineered systemic artificial intelligence     200